# Identificação de Homólogos

In [27]:
import os
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from scripts_python import BLASTer

a = %pwd
wd = (a.rsplit('/',2))[0]
seq_id = "NM_004335"
prot_seq_id = "Q10589"
gene = "BST2"
file_blast_ncbi = "00D0DT8J016-Alignment.xml"
file_blast_uniprot = "B20210131216DA2B77BFBD2E6699CA9B6D1C41EB200F810O.fasta"


## Carregar ficheiro XML da pesquisa

In [28]:
result_handle= open(os.path.join( wd,"data/homologue", gene, file_blast_ncbi))
blast_ncbi = NCBIXML.read(result_handle)
result_handle.close()

print('Foram carregadas {0} seq.'.format(len(blast_ncbi.alignments)))

Foram carregadas 100 seq.


### Resumo

In [29]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import re

query_len = blast_ncbi.query_length
homologue = []
sources = {}

print("Accession", "Identity%")
for aln in blast_ncbi.alignments:
    for HSP in aln.hsps:
        if HSP.score > 80 and HSP.expect < 1.0e-10 and HSP.identities/HSP.align_length > 0.80:
            print(aln.accession,HSP.identities/HSP.align_length)
            species = re.findall('\[[^\]]*\]', aln.hit_def)[0].replace("[","").replace("]","")
            if species not in sources:
                sources[species] = 1
            else:
                sources[species] +=1
            homologue.append(SeqRecord(Seq(HSP.sbjct), aln.title.split( ">", 1)[0], "", ""))


print("Total:",len(homologue))

Accession Identity%
NP_004326 1.0
BAD96844 0.9944444444444445
NP_001266147 0.9567567567567568
XP_018869871 0.9513513513513514
NP_001266560 0.9459459459459459
NP_001177409 0.9459459459459459
ADI58596 0.907608695652174
NP_001295602 0.9017341040462428
BAG64608 0.9629629629629629
NP_001166058 0.9127906976744186
ADI58598 0.9190751445086706
XP_030675274 0.9190751445086706
3MQB_A 1.0
3MQC_A 0.9739130434782609
3MQ7_A 0.9478260869565217
3NWH_A 0.963302752293578
2XG7_A 1.0
XP_024206821 0.92
3MQ9_A 0.9318181818181818
2X7A_A 0.9672131147540983
Total: 20


In [30]:
for source in sources.keys():
    print(source, ":", sources[source])

SeqIO.write(homologue, os.path.join(wd,"data/homologue", gene, f"{seq_id}_{gene}.fasta"), "fasta")

Homo sapiens : 9
Pan paniscus : 1
Gorilla gorilla gorilla : 1
Gorilla gorilla : 1
Pan troglodytes : 2
Pongo pygmaeus : 1
Nomascus leucogenys : 2
Pongo abelii : 1
Hylobates agilis : 1
Escherichia coli K-12 : 1


20

### Resultados UNIPROT

#### Comentários

In [31]:
path = os.path.join(wd, "data/homologue", gene, file_blast_uniprot)
blast_uniprot_raw = SeqIO.parse(path, format="fasta")
blast_uniprot = []
blast_uniprot_id = []

for protein in blast_uniprot_raw:
    blast_uniprot.append(protein)
    seq_id = re.findall('\|[^|]*\|', protein.id)[0].replace("|","")
    blast_uniprot_id.append(seq_id)

In [32]:
from wget import download

for accession in blast_uniprot_id:
    url = "https://www.uniprot.org/uniprot/{0}.xml".format(accession)
    path = os.path.join(wd, "data/homologue", gene, "uniprot","{0}.xml".format(accession))
    download(url, path)

In [33]:
import re

blast_matches = []

for accession in blast_uniprot_id:
    path = os.path.join(wd, "data/homologue", gene,"uniprot","{0}.xml".format(accession))
    blast_matches.append(SeqIO.read(path, format="uniprot-xml"))

comments = {}
an_domains = {}
keywords = {}
for match in blast_matches:
    if "keywords" in match.annotations.keys():
        for keyword in match.annotations["keywords"]:
            if keyword in keywords:
                keywords[keyword] = ", ".join([keywords[keyword],match.id])
            else:
                keywords[keyword] = match.id
    for annotation in match.annotations.keys():
        if re.match("(comment_)[a-z]*", annotation):
            comment_type = annotation.split("_",1)[1]
            if comment_type not in comments.keys():
                comments[comment_type] = {}
            comment = match.annotations[annotation][0]
            if comment in comments[comment_type].keys():
                comments[comment_type][comment] = ", ".join([comments[comment_type][comment], match.id])
            else:
                 comments[comment_type][comment] = match.id

for comment_type in comments.keys():
    print(comment_type)
    for comment in comments[comment_type].keys():
        print(comment, ":", comments[comment_type][comment])
    print("\n")

print("Keywords")
for keyword in keywords.keys():
    print(keyword, ":", keywords[keyword])
print("\n")

function
IFN-induced antiviral host restriction factor which efficiently blocks the release of diverse mammalian enveloped viruses by directly tethering nascent virions to the membranes of infected cells. Acts as a direct physical tether, holding virions to the cell membrane and linking virions to each other. The tethered virions can be internalized by endocytosis and subsequently degraded or they can remain on the cell surface. In either case, their spread as cell-free virions is restricted (PubMed:22520941, PubMed:21529378, PubMed:20940320, PubMed:20419159, PubMed:20399176, PubMed:19879838, PubMed:19036818, PubMed:18342597, PubMed:18200009). Its target viruses belong to diverse families, including retroviridae: human immunodeficiency virus type 1 (HIV-1), human immunodeficiency virus type 2 (HIV-2), simian immunodeficiency viruses (SIVs), equine infectious anemia virus (EIAV), feline immunodeficiency virus (FIV), prototype foamy virus (PFV), Mason-Pfizer monkey virus (MPMV), human T-

### Features

In [34]:
features = {}
for match in blast_matches:
    for feature in match.features:
        if re.search("(domain|motif|bond|region)", feature.type):
            feature_type = feature.type
            if feature_type not in features.keys():
                features[feature_type] = {}

            feature_desc = str(feature.location)
            if "description" in feature.qualifiers.keys():
                feature_desc = " ".join([feature_desc,feature.qualifiers["description"]])

            if feature_desc in features[feature_type].keys():
                features[feature_type][feature_desc] = ", ".join([features[feature_type][feature_desc], match.id])
            else:
                 features[feature_type][feature_desc] = match.id

for feature_type in features.keys():
    print(feature_type)
    for feature in features[feature_type].keys():
        print(feature, ":", features[feature_type][feature])
    print("\n")

topological domain
[0:20] Cytoplasmic : Q10589
[48:161] Extracellular : Q10589
[0:26] Cytoplasmic : Q6WRU0, Q811A2
[47:183] Extracellular : Q6WRU0
[0:30] Cytoplasmic : Q8R2Q8
[51:152] Extracellular : Q8R2Q8
[47:152] Extracellular : Q811A2


transmembrane region
[20:48] Helical; Signal-anchor for type II membrane protein : Q10589
[26:49] Helical : G3RGZ0, A0A2R9B2M4, D7RVC2, A0A2K5ZRD2, A0A0D9QZX5, A0A2K5NPL7, A0A2K6JVP3, A0A2K6R756, A0A2K5U138, E3P8S8, A0A0A0MVP3, A0A1D5Q5T5, A0A2K5KE46, K7ETD2, G1PWT4, A0A287BJ49, G5B7V8, S7N8H4
[28:52] Helical : F7IEG2, G1LVF2, A0A452QB38, A0A3Q7V4T5, A0A384D602, S7N5S0, A0A3Q7RJQ5, S7NIJ0
[26:51] Helical : G1R1B1, A0A2K5PGD0, A0A1U7U677
[26:52] Helical : A0A2K5DGV6, A0A2K6TMV5, A0A5F4VSG4, G1Q1P5
[19:43] Helical : A0A2Y9RLS3, G3UKK9
[27:51] Helical : L9KXG5, A0A1A6FWS4, A0A1S3WP28
[163:189] Helical : L9KXG5
[31:54] Helical : A0A2U3YUR1, G3N1J6, F1N713, L8HPY7
[29:51] Helical : A0A2Y9GI37, G1PEX9, L5L6U1, M3YWS1, R7J3C3
[165:184] Helical : A0A2Y9GI37